In [1]:

from openff.toolkit.topology import Molecule
import parmed
from openff.evaluator import unit
from openff.evaluator.substances import Substance, Component, MoleFraction
from openff.evaluator.workflow.schemas import WorkflowSchema

from openff.evaluator.protocols.coordinates import BuildCoordinatesPackmol
from openff.evaluator.protocols.forcefield import BuildSmirnoffSystem
from openff.evaluator.workflow.utils import ProtocolPath

from openff.evaluator.thermodynamics import ThermodynamicState

from openff.evaluator.workflow import Workflow



Warning on use of the timeseries module: If the inherent timescales of the system are long compared to those being analyzed, this statistical inefficiency may be an underestimate.  The estimate presumes the use of many statistically independent samples.  Tests should be performed to assess whether this condition is satisfied.   Be cautious in the interpretation of the data.

****** PyMBAR will use 64-bit JAX! *******
* JAX is currently set to 32-bit bitsize *
* which is its default.                  *
*                                        *
* PyMBAR requires 64-bit mode and WILL   *
* enable JAX's 64-bit mode when called.  *
*                                        *
* This MAY cause problems with other     *
* Uses of JAX in the same code.          *
******************************************



In [2]:

def parameterize_system(smiles, force_field_path="openff_unconstrained-2.1.0.offxml", mass_density=0.95 * unit.grams / unit.milliliters):
    # Define the substance
    substance = Substance()
    substance.add_component(Component(smiles=smiles), MoleFraction(1.0))

    # Create the workflow schema
    schema = WorkflowSchema()

    # Step 1: Build coordinates
    build_coordinates = BuildCoordinatesPackmol("build_coordinates")
    build_coordinates.max_molecules = 1000
    build_coordinates.mass_density = mass_density
    build_coordinates.substance = substance
    schema.protocol_schemas.append(build_coordinates.schema)

    # Step 2: Assign parameters
    assign_parameters = BuildSmirnoffSystem("assign_parameters")
    assign_parameters.force_field_path = force_field_path
    assign_parameters.coordinate_file_path = ProtocolPath("coordinate_file_path", build_coordinates.id)
    assign_parameters.substance = substance
    schema.protocol_schemas.append(assign_parameters.schema)

    # Metadata
    metadata = {
        "substance": substance,
        "thermodynamic_state": ThermodynamicState(
            temperature=298.15 * unit.kelvin,
            pressure=1.0 * unit.atmosphere
        ),
        "force_field_path": force_field_path
    }

    # Create and execute the workflow
    workflow = Workflow.from_schema(schema, metadata=metadata)
    workflow.execute()

    return workflow


In [3]:
smiles = 'ClC(Cl)Cl'
workflow = parameterize_system(smiles)
print(smiles,workflow.schema.id)

ClC(Cl)Cl 4a6a781c2b774a15ad79d7fc00fb2ebf
